<center>
<h1>2. Supervised Modelling</h1>
</center>

<br>

<center>
    <b>ALEXANDER SCHINDLER</b>
  <p style="font-size:9pt; text-align: center;color: gray;">  
Scientist<br>
Information Management<br>
Center for Digital Safety & Security<br>
<br>
      <b>AIT Austrian Institute of Technology GmbH</b><br>
Giefinggasse 4 | 1210 Vienna | Austria<br>
T +43 50550-2902 | M +43 664 8251454 | F +43 50550-2813<br>
alexander.schindler@ait.ac.at | www.ait.ac.at<br>
    </p>
</center>

# Imports

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.cm as cm

import os
import glob

import numpy as np
import pandas as pd

import cv2

import progressbar

from IPython.display import Image

In [3]:
DATA_PATH            = "/datastorage_fast/FIDS30_all"
IMAGES_PATH          = "/datastorage_fast/FIDS30_all"
IMAGES_LABELLED_PATH = "/datastorage_fast/FIDS30"

store_dir            = "/datastorage_fast/FIDS30_temp"

# Metadata

In [4]:
filenames = []
labels    = []

for path, subdirs, files in os.walk(IMAGES_PATH):
    for name in files:
        if name.find(".jpg") != -1:
            filenames.append(os.path.join(path, name))
            labels.append(name.split("_")[0])
        
metadata_images          = pd.DataFrame(filenames, columns=["Filename"])
metadata_images["label"] = labels
metadata_images.head()

,Filename,label
0,/datastorage_fast/FIDS30_all/lemons_37.jpg,lemons
1,/datastorage_fast/FIDS30_all/blackberries_7.jpg,blackberries
2,/datastorage_fast/FIDS30_all/mangos_23.jpg,mangos
3,/datastorage_fast/FIDS30_all/guava_32.jpg,guava
4,/datastorage_fast/FIDS30_all/bananas_6.jpg,bananas


# Load Images

## Preprocessing Functions

In [5]:
def resize_image(img, height, width):
    
    is_color              = len(img.shape) == 3
    src_height, src_width = img.shape[:2]
    
    # calc ratio
    ratio = np.min([float(height) / src_height, float(width) / src_width])
    
    new_height = np.round(src_height * ratio).astype(int)
    new_width  = np.round(src_width  * ratio).astype(int)
    
    img_reshaped = cv2.resize(img, (new_width,new_height), interpolation=cv2.INTER_CUBIC)

    if new_height < height:
        
        if is_color:
            dims = (height - new_height, new_width, 3)
        else:
            dims = (height - new_height, new_width)
        
        padding      = np.ones(dims, dtype=np.uint8)
        padding     *= 255
        img_reshaped = np.concatenate([img_reshaped, padding], axis=0)
    
    elif new_width < width:
        
        if is_color:
            dims = (new_height, width - new_width, 3)
        else:
            dims = (new_height, width - new_width)
        
        padding      = np.ones(dims, dtype=np.uint8)
        padding     *= 255

        img_reshaped = np.concatenate([img_reshaped, padding], axis=1)
        
    if not ((img_reshaped.shape[0] == height) & (img_reshaped.shape[1] == width)):
        print(img_reshaped.shape)
        raise Exception("resize error")
    
    return img_reshaped, ratio

## Load Images

In [6]:
pbar = progressbar.ProgressBar(max_value=metadata_images.shape[0])

document_images     = []
image_metadata_idx  = []
failed_idx          = []

for i, f_name in pbar(enumerate(metadata_images.Filename.values)):
    
    try:

        img = cv2.imread(f_name)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img, _ = resize_image(img, height=128, width=128)
        img = img.astype(np.float32)
        img = img / 255.        
        
        document_images.append(img)
        image_metadata_idx.append(i)
        
    except:
        failed_idx.append(i)

document_images = np.array(document_images)

100% (971 of 971) |######################| Elapsed Time: 0:00:51 Time:  0:00:51


In [7]:
metadata_images.loc[image_metadata_idx, "image_nr"] = np.arange(len(image_metadata_idx))

In [8]:
metadata_images = metadata_images[~metadata_images.image_nr.isna()]

In [9]:
metadata_images.head()

,Filename,label,image_nr
0,/datastorage_fast/FIDS30_all/lemons_37.jpg,lemons,0.0
1,/datastorage_fast/FIDS30_all/blackberries_7.jpg,blackberries,1.0
2,/datastorage_fast/FIDS30_all/mangos_23.jpg,mangos,2.0
4,/datastorage_fast/FIDS30_all/bananas_6.jpg,bananas,3.0
5,/datastorage_fast/FIDS30_all/tomatoes_21.jpg,tomatoes,4.0


# Model: Convolutional Neural Networks


In [10]:
from keras.models import *
from keras.optimizers import *
from keras.regularizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
from keras.constraints import *
from keras.layers import *

Using TensorFlow backend.


## Train Model

In [11]:
# define inputs
inputs = Input((128, 128, 3))

# normalize inputs
inp = BatchNormalization()(inputs)

# === cnn ===

activation = 'relu'
filtnr     = 12

cnn = Conv2D(filtnr, 3, activation = activation, padding = 'same')(inp)
cnn = MaxPooling2D(pool_size=(2, 2), padding='same')(cnn) # 64x64
cnn = BatchNormalization()(cnn)
#cnn = Dropout(0.2)(cnn)

cnn = Conv2D(filtnr*2, 3, activation = activation, padding = 'same')(cnn)
cnn = MaxPooling2D(pool_size=(2, 2), padding='same')(cnn) # 32x32
cnn = BatchNormalization()(cnn)
#cnn = Dropout(0.2)(cnn)

cnn = Conv2D(filtnr*4, 3, activation = activation, padding = 'same')(cnn)
cnn = MaxPooling2D(pool_size=(2, 2), padding='same')(cnn) # 16x16
cnn = BatchNormalization()(cnn)
#cnn = Dropout(0.2)(cnn)

cnn = Conv2D(filtnr*8, 3, activation = activation, padding = 'same')(cnn)
cnn = MaxPooling2D(pool_size=(2, 2), padding='same')(cnn) # 8x8
cnn = BatchNormalization()(cnn)
#cnn = Dropout(0.2)(cnn)

# === Bottleneck ===
cnn = Flatten()(cnn)

fc  = Dense(256, activation="relu")(cnn)
out = Dense(30,  activation="softmax")(fc)


model         = Model(input = inputs, output = out)

model.compile(optimizer = Adam(lr=0.0002), loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 128, 3)       12        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 12)      336       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 12)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 64, 64, 12)        48        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 24)        2616      
_________________________________________________________________
max_

/home/schindler/anaconda/python2/envs/py36/lib/python3.7/site-packages/ipykernel_launcher.py:39: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


## Prepare ground-truth data

In [12]:
metadata_images.head()

,Filename,label,image_nr
0,/datastorage_fast/FIDS30_all/lemons_37.jpg,lemons,0.0
1,/datastorage_fast/FIDS30_all/blackberries_7.jpg,blackberries,1.0
2,/datastorage_fast/FIDS30_all/mangos_23.jpg,mangos,2.0
4,/datastorage_fast/FIDS30_all/bananas_6.jpg,bananas,3.0
5,/datastorage_fast/FIDS30_all/tomatoes_21.jpg,tomatoes,4.0


In [13]:
from sklearn.preprocessing import LabelEncoder

**Numerically encode string labels**

In [14]:
lencoder = LabelEncoder()
lencoder.fit(metadata_images.label)

labels_encoded = lencoder.transform(metadata_images.label)

**One-Hot Encode labels**

In [15]:
from sklearn.preprocessing import OneHotEncoder

In [16]:
ohe = OneHotEncoder(sparse=False)
ohe.fit(labels_encoded.reshape(-1, 1))

labels_ohe = ohe.transform(labels_encoded.reshape(-1, 1))

/home/schindler/anaconda/python2/envs/py36/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [17]:
labels_ohe

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Prepare Evaluation

Train / test

In [18]:
from sklearn.model_selection import StratifiedShuffleSplit

In [19]:
sss_tt = StratifiedShuffleSplit(n_splits=1, test_size=0.1)

In [20]:
tt_splits = sss_tt.split(document_images, labels_encoded)

In [21]:
train_all_ids, test_ids = list(tt_splits)[0]

Train / Validation

In [22]:
sss_tv = StratifiedShuffleSplit(n_splits=1, test_size=0.05)

In [23]:
tv_splits = sss_tv.split(document_images[train_all_ids], labels_encoded[train_all_ids])

In [24]:
train_ids_rel, val_ids_rel = list(tv_splits)[0]

In [25]:
train_ids = train_all_ids[train_ids_rel]
val_ids   = train_all_ids[val_ids_rel]

## Train Model

In [26]:
from keras.callbacks import TensorBoard

In [27]:
model_nr = 0

In [28]:
model_nr += 1

tb_callback = TensorBoard(log_dir='/datastorage_fast/FIDS30_temp/%d' % model_nr, 
                                      histogram_freq=0, 
                                      batch_size=32, 
                                      write_graph=False, 
                                      write_grads=False, 
                                      write_images=False, 
                                      embeddings_freq=0, 
                                      embeddings_layer_names=None, 
                                      embeddings_metadata=None, 
                                      embeddings_data=None, 
                                      update_freq='epoch')

In [29]:
model.fit(document_images[train_ids], 
          labels_ohe[train_ids], 
          batch_size = 64,
          epochs     = 50,
          validation_data = (document_images[val_ids], labels_ohe[val_ids]),
          callbacks=[tb_callback],
          verbose    = 1)

Instructions for updating:
Use tf.cast instead.
Train on 822 samples, validate on 44 samples
Epoch 1/50
822/822 [==============================] - 4s 5ms/step - loss: 3.3846 - acc: 0.1277 - val_loss: 2.9712 - val_acc: 0.1818
Epoch 2/50
822/822 [==============================] - 0s 529us/step - loss: 1.5070 - acc: 0.5998 - val_loss: 2.5703 - val_acc: 0.2500
Epoch 3/50
822/822 [==============================] - 0s 573us/step - loss: 0.7228 - acc: 0.8589 - val_loss: 2.5479 - val_acc: 0.2727
Epoch 4/50
822/822 [==============================] - 0s 545us/step - loss: 0.3451 - acc: 0.9696 - val_loss: 2.4486 - val_acc: 0.3182
Epoch 5/50
822/822 [==============================] - 0s 504us/step - loss: 0.1560 - acc: 0.9951 - val_loss: 2.4323 - val_acc: 0.3409
Epoch 6/50
822/822 [==============================] - 0s 557us/step - loss: 0.0818 - acc: 1.0000 - val_loss: 2.4159 - val_acc: 0.2500
Epoch 7/50
822/822 [==============================] - 0s 581us/step - loss: 0.0544 - acc: 1.0000 - val_lo

# Data Augmentation

In [30]:
from keras.preprocessing.image import ImageDataGenerator

In [31]:
datagen_train = ImageDataGenerator(featurewise_center=False, 
                                   samplewise_center=False, 
                                   featurewise_std_normalization=False, 
                                   samplewise_std_normalization=False, 
                                   zca_whitening=False, 
                                   zca_epsilon=1e-06, 
                                   rotation_range=90, 
                                   width_shift_range=10.0, 
                                   height_shift_range=10.0, 
                                   brightness_range=None, 
                                   shear_range=0.0, 
                                   zoom_range=10.0, 
                                   channel_shift_range=0.0, 
                                   fill_mode='constant', 
                                   cval=1.0, 
                                   horizontal_flip=True, 
                                   vertical_flip=True, 
                                   rescale=None, 
                                   preprocessing_function=None, 
                                   data_format=None, 
                                   validation_split=0.0, 
                                   dtype=None)

In [32]:
datagen_train.fit(document_images[train_ids])

In [33]:
datagen_val = ImageDataGenerator(featurewise_center=False, 
                                 samplewise_center=False, 
                                 featurewise_std_normalization=False, 
                                 samplewise_std_normalization=False, 
                                 zca_whitening=False, 
                                 zca_epsilon=1e-06, 
                                 rotation_range=0, 
                                 width_shift_range=0.0, 
                                 height_shift_range=0.0, 
                                 brightness_range=None, 
                                 shear_range=0.0, 
                                 zoom_range=0.0, 
                                 channel_shift_range=0.0, 
                                 fill_mode='constant', 
                                 cval=1.0, 
                                 horizontal_flip=False, 
                                 vertical_flip=False, 
                                 rescale=None, 
                                 preprocessing_function=None, 
                                 data_format=None, 
                                 validation_split=0.0, 
                                 dtype=None)

In [34]:
datagen_val.fit(document_images[val_ids])

In [35]:
model_aug_nr = 0

In [36]:
model_aug_nr += 1

tb_callback = TensorBoard(log_dir='/datastorage_fast/FIDS30_temp/aug_%d' % model_aug_nr, 
                                      histogram_freq=0, 
                                      batch_size=32, 
                                      write_graph=False, 
                                      write_grads=False, 
                                      write_images=False, 
                                      embeddings_freq=0, 
                                      embeddings_layer_names=None, 
                                      embeddings_metadata=None, 
                                      embeddings_data=None, 
                                      update_freq='epoch')

In [37]:
model.fit_generator(datagen_train.flow(document_images[train_ids], labels_ohe[train_ids], batch_size=64),
                    validation_data = datagen_val.flow(document_images[val_ids], labels_ohe[val_ids], batch_size=12),
                    steps_per_epoch = train_ids.shape[0] / 64, 
                    validation_steps= val_ids.shape[0] / 12,
                    callbacks=[tb_callback],
                    verbose=1,
                    epochs=50)

Epoch 1/50
13/12 [==============================] - 3s 222ms/step - loss: 4.1325 - acc: 0.0730 - val_loss: 3.4724 - val_acc: 0.2273
Epoch 2/50
13/12 [==============================] - 3s 209ms/step - loss: 3.2662 - acc: 0.1093 - val_loss: 3.8758 - val_acc: 0.2045
Epoch 3/50
13/12 [==============================] - 3s 223ms/step - loss: 3.0941 - acc: 0.1478 - val_loss: 3.7732 - val_acc: 0.2500
Epoch 4/50
13/12 [==============================] - 3s 217ms/step - loss: 2.9019 - acc: 0.1616 - val_loss: 3.5949 - val_acc: 0.2500
Epoch 5/50
13/12 [==============================] - 3s 210ms/step - loss: 2.8220 - acc: 0.1586 - val_loss: 3.3943 - val_acc: 0.2727
Epoch 6/50
13/12 [==============================] - 3s 204ms/step - loss: 2.6661 - acc: 0.1879 - val_loss: 3.4003 - val_acc: 0.1818
Epoch 7/50
13/12 [==============================] - 3s 210ms/step - loss: 2.6294 - acc: 0.2054 - val_loss: 3.1814 - val_acc: 0.2045
Epoch 8/50
13/12 [==============================] - 3s 198ms/step - loss: 2.

In [38]:
model.evaluate(document_images[test_ids], labels_ohe[test_ids])

97/97 [==============================] - 0s 685us/step


[2.384141274334229, 0.29896907216494845]